# Propositionalization: Interstate 94

In this notebbok, we compare getML's FastProp against well-known feature engineering libraries featuretools and tsfresh.

Summary:

- Prediction type: __Regression model__
- Domain: __Transportation__
- Prediction target: __Hourly traffic volume__ 
- Source data: __Multivariate time series, 5 components__
- Population size: __24096__

_Author: Sören Nikolaus_

# Background

A common approach to feature engineering is to generate attribute-value representations from relational data by applying a fixed set of aggregations to columns of interest and perform a feature selection on the (possibly large) set of generated features afterwards. In academia, this approach is called _propositionalization._

getML's [FastProp](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#fastprop) is an implementation of this propositionalization approach that has been optimized for speed and memory efficiency. In this notebook, we want to demonstrate how – well – fast FastProp is. To this end, we will benchmark FastProp against the popular feature engineering libraries [featuretools](https://www.featuretools.com/) and [tsfresh](https://tsfresh.readthedocs.io/en/latest/). Both of these libraries use propositionalization approaches for feature engineering.

In this notebook, we predict the hourly traffic volume on I-94 westbound from Minneapolis-St Paul. The analysis is built on top of a dataset provided by the [MN Department of Transportation](https://www.dot.state.mn.us), with some data preparation done by [John Hogue](https://github.com/dreyco676/Anomaly_Detection_A_to_Z/). For further details about the data set refer to [the full notebook](../interstate94.ipynb).

# Analysis

Let's get started with the analysis and set-up your session:

In [1]:
import datetime
import os
import time

import numpy as np
import pandas as pd

from IPython.display import Image
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline  

import getml

from utils import FTTimeSeriesBuilder, TSFreshBuilder

print(f"getML API version: {getml.__version__}\n")

getml.engine.set_project('interstate94')

getML API version: 1.0.0



Connected to project 'interstate94'


## 1. Loading data

### 1.1 Download from source

We begin by downloading the data from the UC Irvine Machine Learning repository:

In [2]:
data = getml.datasets.load_interstate94(roles=True, units=True)
traffic_test, traffic_train, _ = data.values()



Loading traffic_test...


Loading traffic_train...


Loading weather...


In [3]:
traffic, split = getml.data.split.concat("traffic", train=traffic_train, test=traffic_test)

In [4]:
for data_frame in [traffic_test, traffic_train, traffic]:
    data_frame.set_role(data_frame.roles.categorical, getml.data.roles.unused_string)

In [5]:
traffic

name,ds,join_key,Traffic_Volume,ds_day,lower_window,upper_window,holiday,hour,weekday,day,month,year
role,time_stamp,join_key,target,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string
unit,time stamp,,,,,,,hour,weekday,day,month,year
0,2016-01-01,1,1513,2016-01-01 00:00:00,-1.0,0.0,New Years Day,0,4,1,1,2016
1,2016-01-01 01:00:00,1,1550,2016-01-01 00:00:00,-1.0,0.0,New Years Day,1,4,1,1,2016
2,2016-01-01 02:00:00,1,993,2016-01-01 00:00:00,-1.0,0.0,New Years Day,2,4,1,1,2016
3,2016-01-01 03:00:00,1,719,2016-01-01 00:00:00,-1.0,0.0,New Years Day,3,4,1,1,2016
4,2016-01-01 04:00:00,1,533,2016-01-01 00:00:00,-1.0,0.0,New Years Day,4,4,1,1,2016
,...,...,...,...,...,...,...,...,...,...,...,...
24091,2018-09-30 19:00:00,1,3543,2018-09-30 00:00:00,,,No holiday,19,6,30,9,2018
24092,2018-09-30 20:00:00,1,2781,2018-09-30 00:00:00,,,No holiday,20,6,30,9,2018


### 1.2 Define relational model

To start with relational learning, we need to specify the data model. We manually replicate the appropriate time series structure by setting time series related join conditions (`horizon`, `memory` and `allow_lagged_targets`). This is done abstractly using [Placeholders](https://docs.getml.com/latest/user_guide/data_model/data_model.html#placeholders)

The data model consists of two tables:
* __Population table__ `traffic_{test/train}`: holds target and the contemporarily available time-based components
* __Peripheral table__ `traffic`: same table as the population table
* Join between both placeholders specifies (`horizon`) to prevent leaks and (`memory`) that keeps the computations feasible

In [6]:
time_series = getml.data.TimeSeries(
    population=traffic,
    split=split,
    alias="traffic",
    on='join_key',
    time_stamps='ds',
    horizon=getml.data.time.hours(1),
    memory=getml.data.time.hours(24),
    lagged_targets=True,
)

time_series

data model

  traffic:
    columns:
    - join_key: join_key
    - Traffic_Volume: target
    - ds: time_stamp
    - ds_day: unused_string
    - lower_window: unused_string
    - ...

    joins:
    - right: 'traffic'
      on: (traffic.join_key, traffic.join_key)
      time_stamps: (traffic.ds, traffic.ds)
      relationship: 'many-to-many'
      memory: 86400.0
      horizon: 3600.0
      lagged_targets: True

  traffic:
    columns:
    - join_key: join_key
    - Traffic_Volume: target
    - ds: time_stamp
    - ds_day: unused_string
    - lower_window: unused_string
    - ...


container

  population
      subset   name       rows   type
  0   test     traffic    4819   View
  1   train    traffic   19277   View

  peripheral
      name       rows   type     
  0   traffic   24096   DataFrame

## 2. Predictive modeling

We loaded the data, defined the roles, units and the abstract data model. Next, we create a getML pipeline for relational learning.

### 2.1 Propositionalization with getML's FastProp

In [7]:
seasonal = getml.preprocessors.Seasonal()

fast_prop = getml.feature_learning.FastProp(
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,
)

__Build the pipeline__

In [8]:
pipe_fp_fl = getml.pipeline.Pipeline(
    preprocessors=[seasonal],
    feature_learners=[fast_prop],
    data_model=time_series.data_model,
    tags=["feature learning", "fastprop"],
)

pipe_fp_fl

Pipeline(data_model='traffic',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=['traffic'],
         predictors=[],
         preprocessors=['Seasonal'],
         share_selected_features=0.5,
         tags=['feature learning', 'fastprop'])

In [9]:
pipe_fp_fl.check(time_series.train)

Checking data model...


Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

OK.


In [10]:
begin = time.time()

pipe_fp_fl.fit(time_series.train)

fastprop_train = pipe_fp_fl.transform(time_series.train, df_name="fastprop_train")

end = time.time()

fastprop_runtime = datetime.timedelta(seconds=end - begin)

Checking data model...


Staging...
[========================================] 100%

OK.


Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

FastProp: Trying 461 features...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:24.182341



Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

FastProp: Building features...
[========================================] 100%



In [11]:
fastprop_test = pipe_fp_fl.transform(time_series.test, df_name="fastprop_test")



Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

FastProp: Building features...
[========================================] 100%



In [12]:
predictor = getml.predictors.XGBoostRegressor()

pipe_fp_pr = getml.pipeline.Pipeline(
    tags=["prediction", "fastprop"], predictors=[predictor]
)

In [13]:
pipe_fp_pr.fit(fastprop_train)

Checking data model...


Staging...
[========================================] 100%

OK.


Staging...
[========================================] 100%

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:10.138668



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['prediction', 'fastprop'])

url: http://localhost:1709/#/getpipeline/interstate94/GAaslI/0/

In [14]:
pipe_fp_pr.score(fastprop_test)



Staging...
[========================================] 100%



,date time,set used,target,mae,rmse,rsquared
0,2021-08-07 13:44:15,fastprop_train,Traffic_Volume,202.214,293.855,0.9777
1,2021-08-07 13:44:15,fastprop_test,Traffic_Volume,188.8802,269.7428,0.9819


### 2.2 Propositionalization with featuretools

In [15]:
dfs_pandas = {}

for df in [traffic_train, traffic_test, traffic]:
    dfs_pandas[df.name] = df.to_pandas()
    del dfs_pandas[df.name]["holiday"]
    del dfs_pandas[df.name]["hour"]
    del dfs_pandas[df.name]["day"]
    del dfs_pandas[df.name]["weekday"]
    del dfs_pandas[df.name]["month"]
    del dfs_pandas[df.name]["year"]
    del dfs_pandas[df.name]["ds_day"]
    del dfs_pandas[df.name]["lower_window"]
    del dfs_pandas[df.name]["upper_window"]
    dfs_pandas[df.name]["join_key"] = 1

In [16]:
ft_builder = FTTimeSeriesBuilder(
    num_features=200,
    horizon=pd.Timedelta(hours=1),
    memory=pd.Timedelta(hours=24),
    column_id="join_key",
    time_stamp="ds",
    target="Traffic_Volume",
    allow_lagged_targets=True,
)

In [17]:
featuretools_train = ft_builder.fit(dfs_pandas["traffic_train"])
featuretools_test = ft_builder.transform(dfs_pandas["traffic_test"])

featuretools: Trying features...


/home/patrick/.local/lib/python3.9/site-packages/featuretools/synthesis/dfs.py:307: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['all', 'any', 'entropy', 'num_true', 'percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


Selecting the best out of 77 features...
Time taken: 0h:4m:30.939002



/home/patrick/.local/lib/python3.9/site-packages/featuretools/synthesis/dfs.py:307: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['all', 'any', 'entropy', 'num_true', 'percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


In [18]:
dfs_pandas["traffic_train"]

,ds,join_key,Traffic_Volume
0,2016-01-01 00:00:00,1,1513.0
1,2016-01-01 01:00:00,1,1550.0
2,2016-01-01 02:00:00,1,993.0
3,2016-01-01 03:00:00,1,719.0
4,2016-01-01 04:00:00,1,533.0
...,...,...,...
19271,2018-03-13 23:00:00,1,1287.0
19272,2018-03-14 00:00:00,1,665.0
19273,2018-03-14 01:00:00,1,340.0
19274,2018-03-14 02:00:00,1,285.0


In [19]:
roles={
    getml.data.roles.join_key: ["join_key"],
    getml.data.roles.target: ["Traffic_Volume"],
    getml.data.roles.time_stamp: ["ds"],
}

df_featuretools_train = getml.data.DataFrame.from_pandas(
    featuretools_train, name="featuretools_train", roles=roles
)

df_featuretools_test = getml.data.DataFrame.from_pandas(
    featuretools_test, name="featuretools_test", roles=roles
)

In [20]:
df_featuretools_train.set_role(
    df_featuretools_train.roles.unused, getml.data.roles.numerical
)

df_featuretools_test.set_role(
    df_featuretools_test.roles.unused, getml.data.roles.numerical
)

In [21]:
predictor = getml.predictors.XGBoostRegressor()

pipe_ft_pr = getml.pipeline.Pipeline(
    tags=["prediction", "featuretools"], predictors=[predictor]
)

pipe_ft_pr

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['prediction', 'featuretools'])

In [22]:
pipe_ft_pr.check(df_featuretools_train)

Checking data model...


Staging...
[========================================] 100%

OK.


In [23]:
pipe_ft_pr.fit(df_featuretools_train)

Checking data model...


Staging...
[========================================] 100%

OK.


Staging...
[========================================] 100%

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:4.463031



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['prediction', 'featuretools'])

url: http://localhost:1709/#/getpipeline/interstate94/076RmR/0/

In [24]:
pipe_ft_pr.score(df_featuretools_test)



Staging...
[========================================] 100%



,date time,set used,target,mae,rmse,rsquared
0,2021-08-07 13:50:06,featuretools_train,Traffic_Volume,250.4943,363.6149,0.9658
1,2021-08-07 13:50:06,featuretools_test,Traffic_Volume,241.0873,349.6071,0.9692


### 2.3 Propositionalization with tsfresh

tsfresh failed to run through due to an apparent bug in the tsfresh library and is therefore excluded from this analysis.

### 2.4 Comparison

In [25]:
num_features = dict(
    fastprop=607,
    featuretools=77,
)

runtime_per_feature = [
    fastprop_runtime / num_features['fastprop'],
    ft_builder.runtime / num_features['featuretools'],
]

features_per_second = [1.0/r.total_seconds() for r in runtime_per_feature]

speedup_per_feature = [r/runtime_per_feature[0] for r in runtime_per_feature]

comparison = pd.DataFrame(
    dict(
        runtime=[fastprop_runtime, ft_builder.runtime],
        num_features=num_features.values(),
        features_per_second=features_per_second,
        speedup=[1, ft_builder.runtime/fastprop_runtime],
        speedup_per_feature=speedup_per_feature,
        rsquared=[pipe_fp_pr.rsquared, pipe_ft_pr.rsquared],
        rmse=[pipe_fp_pr.rmse, pipe_ft_pr.rmse],
        mae=[pipe_fp_pr.mae, pipe_ft_pr.mae],
    )
)

comparison.index = ["getML: FastProp", "featuretools"]

In [26]:
comparison

,runtime,num_features,features_per_second,speedup,speedup_per_feature,rsquared,rmse,mae
getML: FastProp,0 days 00:00:33.821094,607,17.947521,1.000000,1.000000,0.981914,269.742838,188.880204
featuretools,0 days 00:04:30.939002,77,0.284197,8.010947,63.151728,0.969201,349.607098,241.087284


In [27]:
comparison.to_csv("comparisons/interstate94.csv")

### Why is FastProp so fast?

First, FastProp hugely benefits from getML's custom-built C++-native in-memory database engine. The engine is highly optimized for working with relational data structures and makes use of information about the relational structure of the data to efficiently store and carry out computations on such data. This matters in particular for time series where we [relate the current observation to a certain number of observations from the past](https://docs.getml.com/latest/user_guide/data_model/data_model.html#time-series): Other libraries have to deal explicitly with this inherent structure of (multivariate) time series; and such explicit transformations are costly, in terms of consumption of both, memory and computational resources. All operations on data stored in getML's engine benefit from implementations in modern C++. Further, we are taking advantage of functional design patterns where all column-based operations are evaluated lazily. So, for example, aggregations are carried out only on rows that matter (taking into account even complex conditions that might span multiple tables in the relational model). Duplicate operations are reduced to a bare minimum by keeping track of the relational data model. In addition to the mere advantage in performance, FastProp, by building on an abstract data model, also has an edge in memory consumption based on the abstract database design, the reliance on efficient storage patterns (utilizing pointers and indices) for concrete data, and by taking advantage of functional design patterns and lazy computations. This allows working with data sets of substantial size even without falling back to distributed computing models.

## 3. Conclusion

You are encouraged to reproduce these results. You will need getML (https://getml.com/product) to do so. You can download it for free.

# Next Steps

This tutorial went showcased another time series application of getML and benchmarked getML against popular time series libraries.

If you are interested in further real-world applications of getML, head back to the [notebook overview](welcome.md) and choose one of the remaining examples.

Here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Feature learning with Multirel](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#multirel)
* [Feature learning with Relboost](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#relboost)

# Get in contact

If you have any question schedule a [call with Alex](https://go.getml.com/meetings/alexander-uhlig/getml-demo), the co-founder of getML, or write us an [email](team@getml.com). Prefer a private demo of getML? Just contact us to make an appointment.